In [166]:
%matplotlib inline

In [167]:
# Storage paths for the Crops and the Panos
pano_file_directory = "/Users/marcus/Desktop/Training_Data/Pano_4096_2048/" 
crop_file_directory = "/Users/marcus/Desktop/Training_Data/Crops_From_4096_2048"

In [175]:
import os
import numpy as np
from skimage import io
from skimage.feature import match_template
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import json
import math

In [176]:
# Parses through the pano json file, and creates Pano Objects 

pano_object_list = []

for dir_name, sub_dir_list, file_list in os.walk(pano_file_directory):
    for file_name in file_list:
        if file_name[-5:] == ".json" and len(file_name) > 20:
            
            new_pano_object = pano_object(file_name, "_4096_2048")
            pano_object_list.append(new_pano_object)

In [177]:
class pano_object():
    
    pano_image_width = 4096
    pano_image_height = 2048
    
    
    def __init__(self, json_file, size_extension = ""):
        self.name = json_file[:-21]
        self.json_path = os.path.join(pano_file_directory, self.name + size_extension + "_label.json")
        self.pano_path = os.path.join(pano_file_directory, self.name + size_extension + ".jpg")
        self.labels = []
        
        with open(self.json_path, 'r') as pano_json:
            label_data = json.load(pano_json)
            for label in label_data[self.name]:
                new_label = label_object(label, self.name)
                self.labels.append(new_label)
        self.label_size = len(self.labels)
    
    def get_pano_image(self):
        pano = io.imread(self.pano_path)[:,:,2]
        return pano
    
    
class label_object():
    
    def __init__(self, label_data, pano_name):
        self.id = label_data['label_id']
        self.type = label_data['label_type']
        self.x = label_data['x']
        self.y = label_data['y']
        self.pano_name = pano_name
        crop_file_name = "{}_._{}_._{}_._{}_._.jpg".format(pano_name, label_data['label_type'], label_data['x'], label_data['y'] )
        self.label_crop_path = os.path.join(crop_file_directory, label_data['label_type'], pano_name[:2], crop_file_name)
        
    def get_label_crop_image(self):
        crop = io.imread(self.label_crop_path)[:,:,2]
        return crop
    
    def get_distance(self, x, y):
        x1 = min(self.x, x)
        x2 = max(self.x, x)
        
        dx = min((x2 - x1), (pano_object.pano_image_width - x2 + x1))
        dy = self.y - y
        
        distance = math.sqrt((dx)**2 + (dy)**2)
        
        return distance
        

In [179]:
def compare_labels(test_label, train_label):
    if test_label.type == train_label.type:
        return True
    else:
        return False

In [180]:
data = np.array(pano_object_list, dtype = np.object)

kfold = KFold(2, False, 4)

for train, test in kfold.split(data):
    for train_pano in data[train]:
        train_pano_image = train_pano.get_pano_image()
        for test_pano in data[test]:
            for label in test_pano.labels:
                print("4")
                template = label.get_label_crop_image()
                result = match_template(train_pano_image, template)
                x_pred, y_pred = np.unravel_index(np.argmax(result), result.shape)[::-1]
                
                for test_label in train_pano.labels:
                    correct_label = compare_labels(test_label, label)
                    print(correct_label)
                    print(test_label.get_distance(x_pred, y_pred))
                    
                    print("({}, {})  to ({}, {})".format(test_label.x, test_label.y, x_pred, y_pred))

4
False
1054.88263930045
(1130.0742321047062, 1289.5384615384614)  to (2161, 1066)
False
1063.9816604048895
(3158.6896167200907, 1435.6923076923076)  to (2161, 1066)
False
151.0846014983705
(2248.843462873937, 1188.923076923077)  to (2161, 1066)
False
1234.434583446576
(954.6896167200908, 1328.0)  to (2161, 1066)
False
469.46206919740564
(1706.6896167200907, 1184.3076923076924)  to (2161, 1066)
False
384.2476092131023
(1795.92038595086, 1185.8461538461538)  to (2161, 1066)
False
255.33832108645223
(2376.228078258552, 1203.3846153846155)  to (2161, 1066)
False
833.4155968204603
(2949.1511551816293, 1336.923076923077)  to (2161, 1066)
4
True
618.6694789982558
(1130.0742321047062, 1289.5384615384614)  to (718, 1751)
True
1685.0731158684725
(3158.6896167200907, 1435.6923076923076)  to (718, 1751)
True
1630.7704238425124
(2248.843462873937, 1188.923076923077)  to (718, 1751)
True
484.7174173300393
(954.6896167200908, 1328.0)  to (718, 1751)
True
1139.582085594431
(1706.6896167200907, 1184.3

True
1034.6211843943097
(1130.0742321047062, 1289.5384615384614)  to (2089, 1678)
True
1096.7902688619433
(3158.6896167200907, 1435.6923076923076)  to (2089, 1678)
True
514.534905822649
(2248.843462873937, 1188.923076923077)  to (2089, 1678)
True
1187.0804714157396
(954.6896167200908, 1328.0)  to (2089, 1678)
True
624.4143847143396
(1706.6896167200907, 1184.3076923076924)  to (2089, 1678)
True
572.8098012911586
(1795.92038595086, 1185.8461538461538)  to (2089, 1678)
True
554.7609685744936
(2376.228078258552, 1203.3846153846155)  to (2089, 1678)
True
925.3072339584905
(2949.1511551816293, 1336.923076923077)  to (2089, 1678)
4
True
1808.8854679487993
(1130.0742321047062, 1289.5384615384614)  to (2594, 227)
True
1334.095745404369
(3158.6896167200907, 1435.6923076923076)  to (2594, 227)
True
1021.9731117001147
(2248.843462873937, 1188.923076923077)  to (2594, 227)
True
1974.725179038674
(954.6896167200908, 1328.0)  to (2594, 227)
True
1305.2807108158072
(1706.6896167200907, 1184.3076923076

False
513.8177571200631
(1130.0742321047062, 1289.5384615384614)  to (1503, 1643)
False
1668.6175673906414
(3158.6896167200907, 1435.6923076923076)  to (1503, 1643)
False
873.194321547496
(2248.843462873937, 1188.923076923077)  to (1503, 1643)
False
632.3521775186363
(954.6896167200908, 1328.0)  to (1503, 1643)
False
501.8845415986351
(1706.6896167200907, 1184.3076923076924)  to (1503, 1643)
False
542.9475035018168
(1795.92038595086, 1185.8461538461538)  to (1503, 1643)
False
977.6445995604213
(2376.228078258552, 1203.3846153846155)  to (1503, 1643)
False
1478.186810411119
(2949.1511551816293, 1336.923076923077)  to (1503, 1643)
4
True
1284.2683906426616
(1130.0742321047062, 1289.5384615384614)  to (2317, 1780)
True
909.389464353234
(3158.6896167200907, 1435.6923076923076)  to (2317, 1780)
True
594.9934810963051
(2248.843462873937, 1188.923076923077)  to (2317, 1780)
True
1435.3374447816282
(954.6896167200908, 1328.0)  to (2317, 1780)
True
852.8353237190968
(1706.6896167200907, 1184.30

True
1131.0251542993904
(1130.0742321047062, 1289.5384615384614)  to (1571, 248)
True
1982.7686039455707
(3158.6896167200907, 1435.6923076923076)  to (1571, 248)
True
1159.6585690828663
(2248.843462873937, 1188.923076923077)  to (1571, 248)
True
1243.47838281919
(954.6896167200908, 1328.0)  to (1571, 248)
True
946.088667493804
(1706.6896167200907, 1184.3076923076924)  to (1571, 248)
True
964.4400387272956
(1795.92038595086, 1185.8461538461538)  to (1571, 248)
True
1249.460692190663
(2376.228078258552, 1203.3846153846155)  to (1571, 248)
True
1756.4322002241022
(2949.1511551816293, 1336.923076923077)  to (1571, 248)
4
True
1503.5887012224255
(1130.0742321047062, 1289.5384615384614)  to (2544, 1801)
True
715.047575314163
(3158.6896167200907, 1435.6923076923076)  to (2544, 1801)
True
679.525967989129
(2248.843462873937, 1188.923076923077)  to (2544, 1801)
True
1658.202790493772
(954.6896167200908, 1328.0)  to (2544, 1801)
True
1039.9029186973235
(1706.6896167200907, 1184.3076923076924)  t

True
1765.0106162190023
(1130.0742321047062, 1289.5384615384614)  to (3530, 1778)
True
505.02074902373164
(3158.6896167200907, 1435.6923076923076)  to (3530, 1778)
True
1410.0970512424385
(2248.843462873937, 1188.923076923077)  to (3530, 1778)
True
1585.8741786157868
(954.6896167200908, 1328.0)  to (3530, 1778)
True
1917.5326098894243
(1706.6896167200907, 1184.3076923076924)  to (3530, 1778)
True
1832.3968689603264
(1795.92038595086, 1185.8461538461538)  to (3530, 1778)
True
1288.942391123684
(2376.228078258552, 1203.3846153846155)  to (3530, 1778)
True
729.3382155062503
(2949.1511551816293, 1336.923076923077)  to (3530, 1778)
4
True
1075.5758792201989
(1130.0742321047062, 1289.5384615384614)  to (2149, 1634)
True
1028.9796222184218
(3158.6896167200907, 1435.6923076923076)  to (2149, 1634)
True
456.1383392505836
(2248.843462873937, 1188.923076923077)  to (2149, 1634)
True
1232.888191041752
(954.6896167200908, 1328.0)  to (2149, 1634)
True
630.7627499740718
(1706.6896167200907, 1184.307

...

True
410.4518985982156
(1130.0742321047062, 1289.5384615384614)  to (1045, 888)
True
2056.5800056318417
(3158.6896167200907, 1435.6923076923076)  to (1045, 888)
True
1240.8842739470767
(2248.843462873937, 1188.923076923077)  to (1045, 888)
True
449.1725340313721
(954.6896167200908, 1328.0)  to (1045, 888)
True
725.0044119837414
(1706.6896167200907, 1184.3076923076924)  to (1045, 888)
True
807.8327533577326
(1795.92038595086, 1185.8461538461538)  to (1045, 888)
True
1368.0773559873212
(2376.228078258552, 1203.3846153846155)  to (1045, 888)
True
1956.354658739978
(2949.1511551816293, 1336.923076923077)  to (1045, 888)
4
True
715.5543356891662
(1130.0742321047062, 1289.5384615384614)  to (1005, 585)
True
2120.435574914695
(3158.6896167200907, 1435.6923076923076)  to (1005, 585)
True
1382.7037437479019
(2248.843462873937, 1188.923076923077)  to (1005, 585)
True
744.7013728104517
(954.6896167200908, 1328.0)  to (1005, 585)
True
922.78818169283
(1706.6896167200907, 1184.3076923076924)  to (1

True
1836.0412784885764
(1130.0742321047062, 1289.5384615384614)  to (3463, 1802)
True
476.2206786939773
(3158.6896167200907, 1435.6923076923076)  to (3463, 1802)
True
1360.1615382944117
(2248.843462873937, 1188.923076923077)  to (3463, 1802)
True
1656.9352187218392
(954.6896167200908, 1328.0)  to (3463, 1802)
True
1861.765277740207
(1706.6896167200907, 1184.3076923076924)  to (3463, 1802)
True
1777.3013255237297
(1795.92038595086, 1185.8461538461538)  to (3463, 1802)
True
1240.731070209747
(2376.228078258552, 1203.3846153846155)  to (3463, 1802)
True
693.0636187969127
(2949.1511551816293, 1336.923076923077)  to (3463, 1802)
4
True
845.7864166790224
(1130.0742321047062, 1289.5384615384614)  to (455, 780)
True
1538.9803786130321
(3158.6896167200907, 1435.6923076923076)  to (455, 780)
True
1839.8620742152914
(2248.843462873937, 1188.923076923077)  to (455, 780)
True
741.6156100419349
(954.6896167200908, 1328.0)  to (455, 780)
True
1315.3674797044587
(1706.6896167200907, 1184.307692307692

True
468.97516082859494
(1130.0742321047062, 1289.5384615384614)  to (891, 1693)
True
1846.3277352977825
(3158.6896167200907, 1435.6923076923076)  to (891, 1693)
True
1448.3895933236274
(2248.843462873937, 1188.923076923077)  to (891, 1693)
True
370.51500277040344
(954.6896167200908, 1328.0)  to (891, 1693)
True
961.3102593493393
(1706.6896167200907, 1184.3076923076924)  to (891, 1693)
True
1037.3456167440495
(1795.92038595086, 1185.8461538461538)  to (891, 1693)
True
1563.8496312944103
(2376.228078258552, 1203.3846153846155)  to (891, 1693)
True
2068.7239278056163
(2949.1511551816293, 1336.923076923077)  to (891, 1693)
4
True
1847.9902827688675
(1130.0742321047062, 1289.5384615384614)  to (2949, 1616)
True
276.5512597431876
(3158.6896167200907, 1435.6923076923076)  to (2949, 1616)
True
820.130401037062
(2248.843462873937, 1188.923076923077)  to (2949, 1616)
True
2014.998239418104
(954.6896167200908, 1328.0)  to (2949, 1616)
True
1315.178062821071
(1706.6896167200907, 1184.307692307692

True
985.0321271265842
(1130.0742321047062, 1289.5384615384614)  to (2044, 1657)
True
1136.446231151364
(3158.6896167200907, 1435.6923076923076)  to (2044, 1657)
True
510.93722725922584
(2248.843462873937, 1188.923076923077)  to (2044, 1657)
True
1137.9095355613392
(954.6896167200908, 1328.0)  to (2044, 1657)
True
580.7032912080992
(1706.6896167200907, 1184.3076923076924)  to (2044, 1657)
True
532.4748272475728
(1795.92038595086, 1185.8461538461538)  to (2044, 1657)
True
562.26542943981
(2376.228078258552, 1203.3846153846155)  to (2044, 1657)
True
960.0770023352442
(2949.1511551816293, 1336.923076923077)  to (2044, 1657)
4
True
1848.39732485506
(1130.0742321047062, 1289.5384615384614)  to (2972, 1135)
True
353.93343568589245
(3158.6896167200907, 1435.6923076923076)  to (2972, 1135)
True
725.1641713522608
(2248.843462873937, 1188.923076923077)  to (2972, 1135)
True
2026.5216955386722
(954.6896167200908, 1328.0)  to (2972, 1135)
True
1266.2707508888693
(1706.6896167200907, 1184.307692307

KeyboardInterrupt: 